[![Open In Colab](https://img.shields.io/badge/Open%20in-Colab-yellow?logo=google-colab)](https://colab.research.google.com/github/BF667/BF-RVC-WebUI/blob/main/bfrvc.ipynb)

# BF RVC Fork WebUI

Simply click `Runtime` in the top navigation bar and `Run all`. Wait for the output of the final cell to show the public Gradio URL and click on it.

In [ ]:
#@title Clone repository
from IPython.display import clear_output
import time

# Clone the repository
!git clone https://github.com/BF667/BF-RVC-WebUI.git HRVC
%cd HRVC

# Simple timer for progress indication
start_time = time.time()
print("Cloning repository...", end="", flush=True)
time.sleep(1)  # Simulate work
elapsed = int(time.time() - start_time)
print(f"\rDone Cloning Repository in {elapsed} seconds")
clear_output(wait=True)
print("Done Cloning Repository")

In [ ]:
#@title Install requirements
from IPython.display import clear_output

# Update pip to a recent version
!pip install --upgrade pip

# Install requirements from file
!pip install -q -r requirements.txt

# Install compatible Gradio versions
!pip install gradio==4.44.0 gradio-client==1.3.0

# Install pyngrok for tunneling
!pip install pyngrok

# Check for sox installation
import shutil
if shutil.which("sox") is None:
    print("Installing sox...")
    !apt-get update -qq && apt-get install -y sox
else:
    print("sox is already installed")

# Install CUDA dependencies (if needed)
try:
    import torch
    if torch.cuda.is_available():
        !pip install onnxruntime-gpu
    else:
        print("No GPU detected, installing CPU version of onnxruntime")
        !pip install onnxruntime
except ImportError:
    print("Torch not installed yet, skipping CUDA setup")

clear_output()
print("Finished Installing Requirements")

In [ ]:
#@title Download MDXNet Vocal Separation
from IPython.display import clear_output

# Run the download models script
!python src/download_models.py

clear_output()
print("Finished Downloading Voice Separation Model")

In [ ]:
#@title Run WebUI
from IPython.display import clear_output
from pyngrok import ngrok
import subprocess

# Set your ngrok authtoken (replace with your own token)
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"  # Get from https://dashboard.ngrok.com/
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start the Gradio app in the background
process = subprocess.Popen(["python", "src/webui.py", "--port", "7860"])

# Create a public URL with ngrok
public_url = ngrok.connect(7860)
print(f"Gradio WebUI is running at: {public_url}")

# Keep the cell running to maintain the tunnel
try:
    process.wait()
except KeyboardInterrupt:
    process.terminate()
    ngrok.disconnect(public_url)
    print("WebUI and ngrok tunnel stopped")